In [33]:
import pandas as pd


import numpy as np
from sklearn import preprocessing


from lib.preprop import *
from lib.geo_to_vector import vectorize_geo
from lib.eda_visualization import *

Loading the dataframe

In [2]:
df = pd.read_csv('df_complete.csv').iloc[:,3:]
df.shape

/tmp/ipykernel_45404/1107331204.py:1: DtypeWarning: Columns (18,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_complete.csv').iloc[:,3:]


(13048, 2868)

Removing duplicated rows

In [3]:
df_no_dups = remove_duplicates(df,['company_name'])
df_no_dups.shape

(10070, 2868)

Sorting all the columns by groups

In [16]:
drop_cols = ['funding stage','products','raised']
cat_cols = ['company_name','company_about', 'founded', 'business model','employees','product stage','status','geographical markets','fund_stage',]
num_cols = ['total_raised','total_rounds', 'investors','ipo_price']
tag_cols = [col for col in df_no_dups.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in df_no_dups.columns if col.startswith('targetmarket_')]
sector_list = [col for col in df_no_dups.columns if col.startswith("sector_")]
target_ind_list = [col  for col in df_no_dups.columns if col.startswith("target_industry_")]
technology_list = [col  for col in df_no_dups.columns if col.startswith("core_technology_")]


<h1>Repairing columns</h1>
<p> Before removing NaN values, we will first identify what values we expect to have in these columns and we will implement it</p>

<h2> Column 'founded'  </h2>
<p>We will change the string representation of founded - "month/year" to an integer "year"</br>
Using REGEX to find the year</p>

In [5]:
# Fixing 'founded' column

print(f'type of "founded" col is : {df_no_dups.founded.dtype}')
print('\nRunning script..')
import re
founded_pattern = r"(\d{4})"
year_list=[]
df_founded = df_no_dups.copy()

for i, val in enumerate(df_founded.founded) :
    year = val.split('/')[-1]
    m = re.search(founded_pattern, year)
    if m:
        year_list.append(int(m.group(0)))
        
        
df_founded['founded'] = year_list


print('\nOperation succesfull!')
print(f'type of "founded" col is : {df_founded.founded.dtype}')

df_founded.shape
        

type of "founded" col is : object

Running script..

Operation succesfull!
type of "founded" col is : int64


(10070, 2868)

<h2> Column 'geographical markets'</h2>
<p>The column contains a string representing the geographical markets the company aims for</br>
The script lib/geo_to_vector.py calculates the precentage of the market and add it to the dataframe. </p>

In [6]:
# Fixing 'geographical markets' column

print("Geographical markets col:")
print(df_founded['geographical markets'].head())
print('----------------------------------------')

print('\nRunning the script...\n')
df_geo_market = vectorize_geo(df_founded, 'l')

print('\n----------------------------------------')
print('Operation succesfull!\n')
print("Geographical percentage col:")
print(df_geo_market['geo_market_per'].head())

num_cols.append('geo_market_per')
cat_cols.remove('geographical markets')
df_geo_market = df_geo_market.drop(['geographical markets'], axis=1)
df_geo_market = df_geo_market.dropna(subset=['company_name'])

Geographical markets col:
0    australia, canada, france, india, united kingd...
1                                                  NaN
2                 canada, mexico, spain, united states
3                                global, united states
4    north america, europe, global, france, germany...
Name: geographical markets, dtype: object
----------------------------------------

Running the script...

shape of df['geo_market_per']: (10070,)

----------------------------------------
Operation succesfull!

Geographical percentage col:
0    0.239983
1         NaN
2    0.068498
3    1.000000
4    1.000000
Name: geo_market_per, dtype: float64


<h1> # Checkpoint : save the new df # </h1>

In [8]:
# df_geo_market.to_csv('cp1.csv')
df_geo_market = pd.read_csv('cp1.csv').iloc[:,1:]
df_geo_market

<h1>Handling NaN values</h1>

<p>First, we will check how many null values are in each column of the non-binary columns</p>

In [19]:
print("missing values:")
cols_to_check = num_cols + cat_cols
for col in cols_to_check:
    s = df_geo_market[col].isnull().sum()
    print(f'\t{col}: {s} missing values') 

missing values:
	total_raised: 5704 missing values
	total_rounds: 5704 missing values
	investors: 5704 missing values
	ipo_price: 9920 missing values
	company_name: 0 missing values
	company_about: 2 missing values
	founded: 0 missing values
	business model: 68 missing values
	employees: 32 missing values
	product stage: 163 missing values
	status: 0 missing values
	fund_stage: 5769 missing values


In [21]:
print(f"num cols: {num_cols}")
print(f"cat cols: {cat_cols}")

num cols: ['total_raised', 'total_rounds', 'investors', 'ipo_price']
cat cols: ['company_name', 'company_about', 'founded', 'business model', 'employees', 'product stage', 'status', 'fund_stage']


In [23]:

print(f'before :{df_geo_market.shape}')
new_df = df_geo_market.drop(drop_cols, axis =1)
new_df = remove_missing_str_val_rows(new_df, cat_cols)
new_df = repair_categorical_missing_vals(new_df, cat_cols)
new_df = repair_numeric_missing_vals_zero(new_df, num_cols)


print(f'after :{new_df.shape}')

before :(10070, 2868)
after :(10070, 2865)


In [24]:
new_df[tag_cols] = new_df[tag_cols].fillna(0)
new_df[targetmarket_cols] = new_df[targetmarket_cols].fillna(0)
new_df[sector_list] = new_df[sector_list].fillna(0)
new_df[target_ind_list] = new_df[target_ind_list].fillna(0)
new_df[technology_list] = new_df[technology_list].fillna(0)

In [25]:
for col in num_cols:
    new_df[col] = conv_to_float(new_df[col])

/home/matanm/Documents/GitHub/startup_classifier/lib/m.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i] = decode_price(val)


In [31]:
print("Checking missing values in non binary columns...\n")

for col in num_cols + cat_cols:
    s = new_df[col].isnull().sum()
    print(f'{col}: {s} missing values') 

Checking missing values in non binary columns...

total_raised: 0 missing values
total_rounds: 0 missing values
investors: 0 missing values
ipo_price: 0 missing values
company_name: 0 missing values
company_about: 0 missing values
founded: 0 missing values
business model: 0 missing values
employees: 0 missing values
product stage: 0 missing values
status: 0 missing values
fund_stage: 0 missing values


In [32]:
bin_cols = tag_cols + targetmarket_cols + sector_list + target_ind_list + technology_list
print("Checking missing values in binary columns...\n")
are_missing = False
for col in bin_cols:
    s = new_df[col].isnull().sum()
    if s!=0:
        are_missing = True
        print(f'{col}: {s} missing values') 
if are_missing is False:
    print("No Nan values in binary columns!")

Checking missing values in binary columns...

No Nan values in binary columns!


<h1> # Checkpoint 2 : save the new df # </h1>


In [27]:
new_df.to_csv('cp2.csv')
# new_df = pd.read_csv('cp2.csv').iloc[:,1:]
# new_df